# KITLV Workshop deel 2


Alle teksten in het corpus dat is aangemaakt voor "Soldaat in Indonesië" zijn verrijkt met POS-tags, met lemmas en met waarden die aangeven of woorden positieve of negatieve connotaties hebben. 

De taalkundige verrijkingen zijn toegevoegd met TreeTagger: http://treetaggerwrapper.readthedocs.io/en/latest/

De waarden ten behoeve van de "sentiment analysis" zijn toegevoegd via de _DuoMan Subjectivity Lexicon_: https://ivdnt.org/taalmaterialen/102-taalmaterialen/2035-tstc-duoman-subjectivitylexicon

De verrijkte bestanden zijn beschikbaar gesteld als TEI-bestanden. Deze gecodeerde bestanden kunnen via de onderstaande code worden gedownload. 


In [ ]:
download_corpus("https://pino.leidenuniv.nl/kitlv/texts/XMLcorpus.zip", "username", "password")

In dit deel van de cursus wordt grotendeels met dezelfde modules gewerkt als in deel 1.

In [ ]:
import re
import os
from os.path import isfile, join , isdir
import string
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import xml.etree.ElementTree as ET
import requests
import zipfile
import math
import os
from os.path import isfile, join , isdir
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from kitlvTdm import *



Op basis van de gecodeerd bestanden kan gerichter worden gezocht naar specifieke woord-categorien in de buurt van een opgegeven zoekterm. De functie _collocationPos()_ is een uitbreiding van de collocatie-analyse die in deel 1 van de workshop is besproken. Naast een 'book' (de tekst waar in moet worden gezicht), de 'searchTerm' (de zoekterm) en de 'distance' (het aantal woorden voor en na de gevonden zoekterm; de omvang van de context) moet in deze functie ook een 'Part of Speech'-categorie worden opgegeven. De zoekactie beperkt zich dan tit woorden in de opgegeven categorie. Een overzicht van alle mogelijke POS-categorieen kan hier worden gevonden: http://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/data/dutch-tagset.txt


In [ ]:
freq = dict()
freq = collocationPos( join( 'XML' , '03391.xml' ) , 'ambonees' , 20 , 'adj' )

## The line below sorts the dictionary by value
sorted_f = sorted( freq , key=lambda x: freq[x])

## Change the value of 'max' to see more results
max = 50
i = 0

for f in reversed( sorted_f ):
    i += 1
    print(f)
    if i == max:
        break
    



De code hieronder kan uit een opgegeven boek alle woorden in een bepaalde grammaticale categorie extraheren. Zoeken met de categorie 'nounprop' ('proper name') kan bijvoorbeeld nuttig zijn bij het opsporen van persoonsnamen of plaatsnamen.  	

In [ ]:

freq = dict()
freq = getWordsByPosTag( join( 'XML' , '03395.xml' ) , 'nounprop' )


for f in sorted(freq):
    print(f)


Aangezien de teksten in het corpus zijn verrijkt met labels die aangeven of woorden een negatieve, neutrale of positieve gevoelswaarde hebben, kunnen verschillende teksten ook op dit vlak met elkaar worden vergeleken. In de code hieronder worden vier teksten geselecteerd. In deze teksten worden de woorden met specifieke gevoelswaarden geteld. De totalen worden grafisch weergegeven in een grafiek, via de _matPlotLib_-module.

In [ ]:

xData = []
yData = []
labels = []

dir = 'XML'

for file in ( '03391.xml' , '03592.xml' , '03399.xml' , '03794.xml' ):
    count = sentimentAnalysis( join( dir , file ) )
    xData.append( count['positive'] / count['all'] )
    yData.append( count['negative'] / count['all'] )
    labels.append( file )

import matplotlib.pyplot as plt

%matplotlib inline
plt.scatter(xData,yData,  color='#0a429b', s=30 , marker="o")

plt.xlabel('positive')
plt.ylabel('negative')
plt.title('Words with positive and negative connotation')

for i in range( 0 , len(labels) ):
    plt.annotate( labels[i] , xy = ( xData[i] , yData[i] ) , xytext=( 0, 10) , textcoords='offset pixels',horizontalalignment='right',verticalalignment='bottom')

plt.show()


De onderstaande code kan worden gebruikt om alle woorden met een positieve connotatie te vinden. Gebruik om alle woorden met een negatieve connotatie te vinden de functie _getNegativeWords()_. 

In [ ]:
taggedWords = getPositiveWords( join( 'XML' , '03592.xml' ) )
                                     
for w in taggedWords:
    print(w)

Word2Vec is een algoritme dat de woorden die voorkomen in een verzameling van teksten kan voorstellen als een verzameling van vectoren. De dimensies van die vectoren zeggen iets over hoe het woord wordt gebruikt. Vectoren met vergelijkbare dimensies zijn vaak ook semantisch verwant. Om met Word2Vec te kunnen werken moet de module _gensim_ worden geinstalleerd. Het model met 'word embeddings' of 'vectoren' is vooraf aangemaakt en is opgeslagen onder de naam "egodocumenten.txt". Dit model moet op de eerste plaats worden ingeleven. Het corpus bestaat uit ca. 30 miljoen woorden, dus het inlezen van dit model kan enige tijd duren.

In [ ]:
import gensim

word2vec_model_file = "egodocumenten.txt"
ed_model = gensim.models.KeyedVectors.load( word2vec_model_file )


Wanneer het Word2Vec model is ingelezen, kunnen de vectoren worden geanalyseerd. Word2Vec kan onder meer worden toegepast om op zoek te gaan naar woorden die op een vergelijkbare manier worden gebruikt. In _gensim_ kan dit via de functie _most_similar()_. Wanneer als parameter van deze functie een woord wordt opgegeven gaat het algoritme, op basis van het aangemaakte model, op zoek naar woorden die op een vergelijkbare manier zijn gebruikt in het corpus. 

In [ ]:
for word in ('java','soldaat','japanner','molukken'):
    if word in ed_model:
        print(word,ed_model.most_similar(word))

Dit laatste blok illustreert hoe er met een specifiek lexicon kan worden gezocht. 

In [ ]:
count = countLexiconWords( join( 'XML' , '03592.xml' ) , 'oorlog.txt' )
print( count )